In [1]:
import pandas as pd
import requests
import defs

In [2]:
session = requests.Session()

In [47]:
def fetch_candles(pair_name, count, granularity):
    url = f"{defs.OANDA_URL}/instruments/{pair_name}/candles"
    params = dict(
        count = count,
        granularity = granularity,
        price = "MBA"
    )
    response = session.get(url, params=params, headers=defs.SECURE_HEADER)
    return response.status_code, response.json()

def get_candles_df(json_response):

    prices = ['mid', 'bid', 'ask']
    ohlc = ['o', 'h', 'l', 'c']
    our_data = []
    for candle in json_response['candles']:
        if candle['complete'] == False:
            continue
        new_dict = {}
        new_dict['time'] = candle['time']
        new_dict['volume'] = candle['volume']
        for price in prices:
            for oh in ohlc:
                new_dict[f"{price}_{oh}"] = candle[price][oh]
        our_data.append(new_dict)
    return pd.DataFrame.from_dict(our_data)

def save_file(candles_df, pair, granularity):
    candles_df.to_csv(f"his_data/{pair}_{granularity}.csv")
    
def create_data(pair, granularity):
    code, json_data = fetch_candles(pair, 624, granularity)
    if code != 200:
        print(pair, "Error",code)
        return
    df = get_candles_df(json_data)
    print(f"{pair} loaded {df.shape[0]} candles from {df.time.min()} to {df.time.max()}")
    save_file(df, pair, granularity)

In [31]:
create_data('EUR_USD','D')

EUR_USD loaded 607 candles from 2019-01-01T22:00:00.000000000Z to 2021-05-03T21:00:00.000000000Z


In [39]:
create_data('USD_CAD','D')

USD_CAD loaded 630 candles from 2019-01-01T22:00:00.000000000Z to 2021-05-03T21:00:00.000000000Z


In [48]:
create_data('USD_JPY','D')

USD_JPY loaded 623 candles from 2019-01-01T22:00:00.000000000Z to 2021-05-03T21:00:00.000000000Z
